## Instrumental Variables
(heavily inspired by Wooldridge ch.15 and the [linearmodels documentation](https://bashtage.github.io/linearmodels/iv/examples/basic-examples.html#Panel-IV))

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Research problem

Suppose we want to estimate the effect of another hour of job training on worker productivity. For the two years 1987 and 1988, consider the simple panel data model

$$
lscrap_{it} = \beta_0 + \beta_{1} hrsemp_{it} + U_i + \epsilon_{it}
$$

To get rid of the unobserved heterogeneities, let's use First Differences:

$$
\Delta_t lscrap_{it} = \beta \Delta_t hrsemp_{it} + \Delta_t\epsilon_{it}
$$

Normally, we would estimate this equation by OLS. But what if $\Delta\epsilon$ is correlated with $hrsemp$?  

For example, a firm might hire more skilled workers, while at the same time reducing the level of job training. In this case, we need an instrumental variable for $hrsemp$.

Generally, such an IV would be hard to find, but we can exploit the fact that some firms received job training grants in 1988. If we assume that grant designation is uncorrelated with $\Delta\epsilon$ (something that is reasonable, because the grants were given at the begining of 1988) then $grant$ is valid as an IV, provided $hrsemp$ and $grant$ are correlated. 

## Load data

In [2]:
from linearmodels.datasets import jobtraining

data = jobtraining.load()
print(jobtraining.DESCR)
data.head()
data = data.where(data.year.isin((1987, 1988)))
data = data.dropna(how="all", axis=0).sort_values(["fcode", "year"])
print(data.describe())
data = data.set_index("fcode")
data = data[["year", "hrsemp", "grant", "scrap", "lscrap"]]


H. Holzer, R. Block, M. Cheatham, and J. Knott (1993), "Are Training Subsidies
Effective? The Michigan Experience," Industrial and Labor Relations Review 46,
625-636.

year                     1987, 1988, or 1989
fcode                    firm code number
employ                   # employees at plant
sales                    annual sales, $
avgsal                   average employee salary
scrap                    scrap rate (per 100 items)
rework                   rework rate (per 100 items)
tothrs                   total hours training
union                    =1 if unionized
grant                    =1 if received grant
d89                      =1 if year = 1989
d88                      =1 if year = 1988
totrain                  total employees trained
hrsemp                   tothrs/totrain
lscrap                   log(scrap)
lemploy                  log(employ)
lsales                   log(sales)
lrework                  log(rework)
lhrsemp                  log(1 + hrsemp)
lscrap_1

In [3]:
set(data.year)

{1987.0, 1988.0}

## Calculate first differences

In [4]:
from statsmodels.api import add_constant

In [5]:
deltas = data.loc[data.year == 1988] - data.loc[data.year == 1987]
deltas = add_constant(deltas, has_constant="add")
deltas = deltas.dropna()
print(deltas.describe())

       const  year     hrsemp      grant      scrap     lscrap
count   45.0  45.0  45.000000  45.000000  45.000000  45.000000
mean     1.0   1.0  10.812321   0.377778  -0.817556  -0.185697
std      0.0   0.0  20.523825   0.490310   2.496392   0.626858
min      1.0   1.0 -19.850180   0.000000 -10.000000  -2.502169
25%      1.0   1.0   0.000000   0.000000  -1.000000  -0.355820
50%      1.0   1.0   1.846154   0.000000  -0.110000  -0.167054
75%      1.0   1.0  15.333330   1.000000   0.090000   0.054067
max      1.0   1.0  80.000000   1.000000   5.000000   2.397895


## Is grant a valid instrument for hrsemp?

In [6]:
from statsmodels.regression.linear_model import OLS
from linearmodels.iv import IV2SLS

In [7]:
mod = OLS(deltas.hrsemp, deltas[["const","grant"]])
print(mod.fit().summary())

                            OLS Regression Results                            
Dep. Variable:                 hrsemp   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.325
Method:                 Least Squares   F-statistic:                     22.23
Date:                Sat, 20 Aug 2022   Prob (F-statistic):           2.56e-05
Time:                        10:36:32   Log-Likelihood:                -189.94
No. Observations:                  45   AIC:                             383.9
Df Residuals:                      43   BIC:                             387.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5806      3.185      0.496      0.6

Yes, $grant$ is highly correlated to $hrsemp$.

## Fit regression model with instrumental variable

* Two-Stage Least Squares (2SLS)
* Main characters in this regression:
    * __Dependent variable__: lscrap
    * __Exogenous variable__: a constant column of 1s for the intercept + any other variables not being instrumentalized
    * __Endogenous variable__: hrsemp
    * __Instrumental variable__: grant

In [112]:
iv_mod = IV2SLS(deltas.lscrap, deltas[["const"]], deltas['hrsemp'], deltas['grant'])
iv_res = iv_mod.fit(cov_type="unadjusted")
print(iv_res)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 lscrap   R-squared:                      0.0159
Estimator:                    IV-2SLS   Adj. R-squared:                -0.0070
No. Observations:                  45   F-statistic:                    3.3464
Date:                Wed, Aug 10 2022   P-value (F-stat)                0.0674
Time:                        23:51:58   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.0327     0.1241    -0.2632     0.7924     -0.2759      0.2106
hrsemp        -0.0142     0.0077    -1.8293     0.06

## Fit OLS model for comparison (WRONG!)

In [113]:
from statsmodels.regression.linear_model import OLS

In [114]:
OLS_mod = IV2SLS(deltas.lscrap, deltas[["const", "hrsemp"]], None, None)
OLS_res = OLS_mod.fit(cov_type="unadjusted")
print(OLS_res)

                            OLS Estimation Summary                            
Dep. Variable:                 lscrap   R-squared:                      0.0619
Estimator:                        OLS   Adj. R-squared:                 0.0401
No. Observations:                  45   F-statistic:                    2.9707
Date:                Wed, Aug 10 2022   P-value (F-stat)                0.0848
Time:                        23:51:59   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.1035     0.1014    -1.0208     0.3073     -0.3023      0.0952
hrsemp        -0.0076     0.0044    -1.7236     0.08

## Compare models

In [115]:
from linearmodels.iv import compare

In [116]:
print(compare({"OLS": OLS_res,"Panel IV": iv_res}, precision='std_errors', stars=True))

                 Model Comparison                 
                                OLS       Panel IV
--------------------------------------------------
Dep. Variable                lscrap         lscrap
Estimator                       OLS        IV-2SLS
No. Observations                 45             45
Cov. Est.                unadjusted     unadjusted
R-squared                    0.0619         0.0159
Adj. R-squared               0.0401        -0.0070
F-statistic                  2.9707         3.3464
P-value (F-stat)             0.0848         0.0674
==================     ============   ============
const                       -0.1035        -0.0327
                           (0.1014)       (0.1241)
hrsemp                     -0.0076*       -0.0142*
                           (0.0044)       (0.0077)
==================== ============== ==============
Instruments                                  grant
--------------------------------------------------

Std. Errors reported in parent

Compare coefficient of $hrsemp$ in both models